# Calculating the similarity matrix and using it to fit an SVM

In [72]:
import numpy as np
import pandas as pd
import utils
import turicreate as tc

In [74]:
data = tc.SFrame({
    'x1': [0,-1,0,0,1,-1,1],
    'x2': [0,0,-1,1,0,1,-1],
    'y': [0,0,0,1,1,1,1]
})
data

x1,x2,y
0,0,0
-1,0,0
0,-1,0
0,1,1
1,0,1
-1,1,1
1,-1,1


### Calculating the similarity matrix

In [75]:
def similarity(x, y):
    return np.exp(-(x[0]-y[0])**2-(x[1]-y[1])**2)

In [81]:
data_with_similarities = tc.SFrame(data)
data_with_similarities

x1,x2,y
0,0,0
-1,0,0
0,-1,0
0,1,1
1,0,1
-1,1,1
1,-1,1


In [82]:
for i in range(len(data)):
    column = []
    for j in range(len(data)):
        first = [data[i]['x1'], data[i]['x2']]
        second = [data[j]['x1'], data[j]['x2']]
        column.append(similarity(first, second))
    data_with_similarities = data_with_similarities.add_column(column, 'Sim'+str(i))
data_with_similarities

Columns:
	x1	int
	x2	int
	y	int
	Sim0	float
	Sim1	float
	Sim2	float
	Sim3	float
	Sim4	float
	Sim5	float
	Sim6	float

Rows: 7

Data:
+----+----+---+---------------------+----------------------+
| x1 | x2 | y |         Sim0        |         Sim1         |
+----+----+---+---------------------+----------------------+
| 0  | 0  | 0 |         1.0         | 0.36787944117144233  |
| -1 | 0  | 0 | 0.36787944117144233 |         1.0          |
| 0  | -1 | 0 | 0.36787944117144233 |  0.1353352832366127  |
| 0  | 1  | 1 | 0.36787944117144233 |  0.1353352832366127  |
| 1  | 0  | 1 | 0.36787944117144233 | 0.01831563888873418  |
| -1 | 1  | 1 |  0.1353352832366127 | 0.36787944117144233  |
| 1  | -1 | 1 |  0.1353352832366127 | 0.006737946999085467 |
+----+----+---+---------------------+----------------------+
+----------------------+----------------------+----------------------+
|         Sim2         |         Sim3         |         Sim4         |
+----------------------+----------------------+----------------------+
| 0.36787944117144233  | 0.36787944117144233  | 0.36787944117144233  |
|  0.1353352832366127  |  0.1353352832366127  | 0.01831563888873418  |
|         1.0          | 0.01831563888873418  |  0.1353352832366127  |
| 0.01831563888873418  |         1.0          |  0.1353352832366127  |
|  0.1353352832366127  |  0.1353352832366127  |         1.0          |
| 0.006737946999085467 | 0.36787944117144233  | 0.006737946999085467 |
| 0.36787944117144233  | 0.006737946999085467 | 0.36787944117144233  |
+----------------------+----------------------+----------------------+
+------------------------+------------------------+
|          Sim5          |          Sim6          |
+------------------------+------------------------+
|   0.1353352832366127   |   0.1353352832366127   |
|  0.36787944117144233   |  0.006737946999085467  |
|  0.006737946999085467  |  0.36787944117144233   |
|  0.36787944117144233   |  0.006737946999085467  |
|  0.006737946999085467  |  0.36787944117144233   |
|          1.0           | 0.00033546262790251185 |
| 0.00033546262790251185 |          1.0           |
+------------------------+------------------------+
[7 rows x 10 columns]

### Fitting a linear classifier to the similarity matrix

In [84]:
svm = tc.logistic_classifier.create(data_with_similarities, target='y')

Logistic regression:

--------------------------------------------------------

Number of examples          : 7

Number of classes           : 2

Number of feature columns   : 9

...

| 2         | 3        | 0.001883     | 1.000000          |

| 3         | 4        | 0.002442     | 1.000000          |

| 4         | 5        | 0.004541     | 1.000000          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

In [85]:
svm.coefficients

name,index,class,value,stderr
(intercept),None,1,2.735390696746358,None
x1,None,1,1.6825739743176509,None
x2,None,1,1.682573974317651,None
Sim0,None,1,-5.07647246343446,None
Sim1,None,1,-4.132982490135503,None
Sim2,None,1,-4.132982490135502,None
Sim3,None,1,1.2672981939725765,None
Sim4,None,1,1.267298193972578,None
Sim5,None,1,3.0043310071304776,None
Sim6,None,1,3.004331007130478,None
